In [1]:
import re
import sys
import imaplib
import os.path

# import email.header
import argparse
from configobj import ConfigObj
from getpass import getpass

In [48]:
def connect(config):
    email_connection = imaplib.IMAP4_SSL(config["imap_server"], config["imap_port"])
    email_connection.login(config["username"], config["password"])
    email_connection.select("inbox")
    print("Successfully connected to " + config["username"] + "@icloud.com inbox")
    return email_connection


def search_emails(email_connection, sender):
    _, data = email_connection.search(None, f'(FROM "{sender}")')
    # Split the email identifiers in an array
    mail_ids = data[0]
    if mail_ids is None:
        return None
    else:
        return mail_ids.split()


# Add the deleted flag to an email
def set_deleted(email_connection, email_uid):
    print(bytes(str(email_uid).strip(), 'ascii'))
    email_connection.uid("STORE", bytes(str(email_uid).strip(), 'ascii'), "+FLAGS", "(\\Deleted)")


def fetch_uid(email_connection, email_id):
    status, uid_string = email_connection.fetch(email_id, "UID")
    uid_str = [str(x, encoding='utf-8') for x in uid_string]
    print("".join(uid_str))
    uid_res = re.search(r"\((UID.*?)\)", uid_str[0])
    return uid_res[1].replace("UID", "") if (uid_res != None) else None


def parse_args():
    parser = argparse.ArgumentParser(
        description="Delete all incoming emails from a sender email address"
    )
    parser.add_argument("--email", help="The sender whose messages should be deleted")
    parser.add_argument(
        "--file",
        help="A file which contains the target emails. Each email should be on a separate line.",
    )
    return parser.parse_args()


def validate_input_email(email):
    return re.match(r"[^@]+@[^@]+\.[^@]+", email)


def verify_cli_args(args):
    if args.email is None and args.file is None:
        print("You need to specify a target email or a file. Use --help for details")
        sys.exit()


def import_emails_from_file(filename):
    if os.path.isfile(filename):
        return open(filename).read().split("\n")
    else:
        print(f"The file {filename} doesn't exist")

In [26]:
# args = parse_args()
# verify_cli_args(args)
args = None

In [4]:
config = ConfigObj("config.ini")

In [5]:
if config["password"] == "":
    config["password"] = getpass("Enter your email password: ")
else:
    print("Using password from config.ini")

Using password from config.ini


In [6]:
email_connection = connect(config)

Successfully connected to boothchin@icloud.com inbox


In [7]:
search_emails(email_connection, "mkt@manning.com")

[b'74909',
 b'64854',
 b'25341',
 b'74913',
 b'82959',
 b'63662',
 b'100622',
 b'49998',
 b'66772',
 b'91069',
 b'43100',
 b'25357',
 b'68818',
 b'29211',
 b'37451',
 b'61644',
 b'24007',
 b'24008',
 b'93094',
 b'24009',
 b'72907',
 b'38730',
 b'63704',
 b'53832',
 b'64910',
 b'100671',
 b'52029',
 b'72932',
 b'25372',
 b'83028',
 b'102729',
 b'77032',
 b'50072',
 b'95180',
 b'59694',
 b'63743',
 b'68880',
 b'24053',
 b'98672',
 b'91161',
 b'66865',
 b'91168',
 b'87106',
 b'61736',
 b'81154',
 b'72997',
 b'100743',
 b'96929',
 b'29266',
 b'70964',
 b'59749',
 b'79135',
 b'73015',
 b'77104',
 b'102814',
 b'98732',
 b'98731',
 b'75080',
 b'65016',
 b'25408',
 b'98738',
 b'95268',
 b'81206',
 b'81207',
 b'73049',
 b'29287',
 b'87171',
 b'91241',
 b'22886',
 b'43620',
 b'25416',
 b'71016',
 b'100800',
 b'24119',
 b'66948',
 b'79189',
 b'73060',
 b'87188',
 b'63776',
 b'102858',
 b'50195',
 b'77162',
 b'28510',
 b'98782',
 b'85192',
 b'25434',
 b'87213',
 b'83179',
 b'50213',
 b'83177',
 b'

In [9]:
TARGET_EMAILS_FILE = "target_email_address.txt"

In [10]:
target_emails = []

target_emails = import_emails_from_file(TARGET_EMAILS_FILE)

# if args.email is None:
#     target_emails = import_emails_from_file(args.file)
# else:
#     target_emails = [args.email]

In [59]:
target_emails

['10play@media.10play.com.au', 'reservationsdarlinghurst@atavola.com.au']

In [60]:
def connect_and_clean(config, target_emails):
    print("Connecting to " + config["username"] + "@icloud.com...")
    email_connection = connect(config)

    total_emails_count = 0
    for target_email in target_emails:
        if not validate_input_email(target_email):
            print(f"The email '{target_email}' is not valid")
            continue

        emails = search_emails(email_connection, target_email)
        if emails is not None:
            emails_count = str(len(emails))
        else:
            emails_count = 0
        total_emails_for_target = int(emails_count)
        total_deleted_emails = 0

        while int(emails_count) > 0:
            print(f"Found {emails_count} email(s) for {target_email}")

            for idx, e in enumerate(emails):
                # The fetching of the email UID is required
                # since the email ID may change between operations
                # as specified by the IMAP standard
                uid = fetch_uid(email_connection, e)
                if uid is None:
                    print(
                        f"Email {str(total_deleted_emails + idx + 1)}/{str(total_emails_for_target)} was not valid"
                    )

                else:
                    print(
                        f"Deleted email {str(total_deleted_emails + idx + 1)}/{str(total_emails_for_target)}"
                    )
                    set_deleted(email_connection, uid)
            # Confirm the deletion of the messages
            email_connection.expunge()

            print(f"Deleted {emails_count} email(s) for {target_email}")
            print("Checking for remaining emails...")

            # Verify if there are any emails left on the server for
            # the target address. This is required to circumvent the
            # chunking of the search results by iCloud
            emails = search_emails(email_connection, target_email)
            if emails is not None:
                emails_count = str(len(emails))
            else:
                emails_count = 0
            total_deleted_emails = total_emails_for_target
            total_emails_for_target += int(emails_count)

        print(
            f"The cleanup for {target_email} was successful. Deleted {str(total_emails_for_target)} email(s)"
        )
        total_emails_count += total_emails_for_target

    print(f"The cleanup was successful. Deleted {str(total_emails_count)} email(s)")

    # Close the mailbox and logout
    email_connection.close()
    email_connection.logout()
    return total_emails_count

In [61]:
total_emails_count = connect_and_clean(config, target_emails)

Connecting to boothchin@icloud.com...
Successfully connected to boothchin@icloud.com inbox
The cleanup for 10play@media.10play.com.au was successful. Deleted 0 email(s)
The cleanup for reservationsdarlinghurst@atavola.com.au was successful. Deleted 0 email(s)
The cleanup was successful. Deleted 0 email(s)


In [63]:
target_emails

['10play@media.10play.com.au', 'reservationsdarlinghurst@atavola.com.au']